# Matching Compustat firm with assignee

### Compustat
- 2010년 상장사 재무정보
- nameprocessing 완료 (지난 프로젝트에서)

### matching 방법
- 이름 기준으로 matching (key가 딱히 없음)
- nameprocessing 결과로 1차 matching
- matching 실패한 assignee 중 num_patent 기준 상위 n%에 대해 manual matching
- match 후 key 부여 (index = key로 두면 될 듯)

### 마무리
- compustat_gvkey_numpat
- assignee_gvkey

### 1. assignee nameprocessing

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

def nameprocessing(s):
    s = str(s)
    
    s = s.replace(".","")
    s = s.replace(",","")    
    s = s.replace(";","")
    s = ''.join(s.split()).lower()
    
    s = s.replace("incorporated","inc")
    s = s.replace("limited","ltd")
    s = s.replace("gmbh","")
    
    rm=['inc','ltd','llc','pte','plc','kft']
    for i in range(0,len(s)):
        if s[-3:] in rm:
            s=s[:-3]
        else:
            break

    return s

In [2]:
df = pd.read_csv('assignee_profile.csv')
df['new_assignee'] = df['assignee'].apply(nameprocessing)
df.to_csv('assignee_new.csv', index=False)
df.iloc[0:5,:]

,assignee,num_patent,new_assignee
0,INTERNATIONAL BUSINESS MACHINES CORPORATION,5614,internationalbusinessmachinescorporation
1,SAMSUNG ELECTRONICS CO LTD,4128,samsungelectronicsco
2,MICROSOFT CORPORATION,3011,microsoftcorporation
3,PANASONIC CORPORATION,2128,panasoniccorporation
4,CANON KABUSHIKI KAISHA,2044,canonkabushikikaisha


### 2. matching

1. assignee를 new_assingee 기준으로 뭉치기
2. compustat과 new_assignee 기준으로 match

In [3]:
a=len(df)
df = df.groupby('new_assignee',as_index=False).sum()
df.sort_values('num_patent', ascending=False, inplace=True)

print a, len(df)

7753 7664


약 100개 정도 더 뭉치기 성공

In [4]:
com = pd.read_csv('compustat_10.csv')[['new_assignee','conm']]
df = df.merge(com, on='new_assignee', how='left', sort=False)
df.index.names = ['PCkey']
df.drop_duplicates(inplace=True)   #I don't know why but there are some duplicates
df.to_csv('patent_compustat_key.csv')

In [5]:
print df['num_patent'].median(), df['num_patent'].mean()

2.0 14.9595722483


### 3. manual processing
- 일단 num_patent가 평균 보다 큰 기업의 경우(14개) matching 시도 (엑셀로)
- median이 2.0이니 흠...

### 4. 마무리 = key랑 join 후 쪼개기

457개 상장사, 전체 특허의 80% 차지

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd

os.chdir('E:/apps')

conm = pd.read_csv('compustat_conm.csv')
compu = pd.read_csv('compustat_10.csv')

compu = compu[['gvkey','conm']]
df = conm.merge(compu, on='conm', how='left', sort=False)
df.drop_duplicates(inplace=True)
len(df)

457

In [2]:
del conm,compu

pckey = pd.read_csv('patent_compustat_key.csv')
df1 = df.merge(pckey, on='conm', how='left', sort=False)
df1.drop_duplicates(inplace=True)
len(df)

457

In [4]:
del df1['PCkey']
df1.iloc[0:20,:]

,conm,gvkey,new_assignee,num_patent
0,INTL BUSINESS MACHINES CORP,6066,internationalbusinessmachinescorporation,5614
1,SAMSUNG ELECTRONICS CO LTD,104604,samsungelectronicsco,4128
2,SAMSUNG ELECTRONICS CO LTD,104604,samsungmobiledisplayco,276
3,SAMSUNG ELECTRONICS CO LTD,104604,samsungledco,24
4,MICROSOFT CORP,12141,microsoftcorporation,3011
5,PANASONIC CORP,7114,panasoniccorporation,2128
6,PANASONIC CORP,7114,matsushitaelectricindustrialco,42
7,PANASONIC CORP,7114,panasoniccorp,1
8,CANON INC,2721,canonkabushikikaisha,2044
9,CANON INC,2721,ocetechnologiesbv,32


In [6]:
# compustat_gvkey_numpat
npat = df1[['gvkey','num_patent']]

npat = npat.groupby('gvkey',as_index=False).sum()
len(npat)

457

In [7]:
df = df.merge(npat, on='gvkey', how='left', sort=False)
df.drop_duplicates(inplace=True)
df.to_csv('compustat_gvkey_numpat.csv',index=False)

In [9]:
# assignee_gvkey

del df1['num_patent']

ass = pd.read_csv('assignee_new.csv')
df1 = df1.merge(ass, on='new_assignee', how='left', sort=False)
df1.drop_duplicates(inplace=True)
len(df1)

675

In [10]:
df1.iloc[0:20,:]

,conm,gvkey,new_assignee,assignee,num_patent
0,INTL BUSINESS MACHINES CORP,6066,internationalbusinessmachinescorporation,INTERNATIONAL BUSINESS MACHINES CORPORATION,5614
1,SAMSUNG ELECTRONICS CO LTD,104604,samsungelectronicsco,SAMSUNG ELECTRONICS CO LTD,4128
2,SAMSUNG ELECTRONICS CO LTD,104604,samsungmobiledisplayco,SAMSUNG MOBILE DISPLAY CO LTD,276
3,SAMSUNG ELECTRONICS CO LTD,104604,samsungledco,SAMSUNG LED CO LTD,24
4,MICROSOFT CORP,12141,microsoftcorporation,MICROSOFT CORPORATION,3011
5,PANASONIC CORP,7114,panasoniccorporation,PANASONIC CORPORATION,2128
6,PANASONIC CORP,7114,matsushitaelectricindustrialco,MATSUSHITA ELECTRIC INDUSTRIAL CO LTD,42
7,PANASONIC CORP,7114,panasoniccorp,PANASONIC CORP,1
8,CANON INC,2721,canonkabushikikaisha,CANON KABUSHIKI KAISHA,2044
9,CANON INC,2721,ocetechnologiesbv,OCE TECHNOLOGIES BV,32


In [12]:
df1 = df1[['assignee','gvkey']]
df1.to_csv('assignee_gvkey.csv',index=False)